# 2.1 Undirected point graph via lines

Unlike lines, points do not touch each other and so have to be joined *via* lines to create their graph.

## 0. Libraries and settings

In [ ]:
# Libraries
from datetime import datetime
import geopandas as gpd
import networkx as nx
import pandas as pd
import shapely

#Settings
gpd.options.io_engine = "pyogrio"

print("Last run:", datetime.now())
print("GeoPandas version:", gpd.__version__)
print("NetworkX version:", nx.__version__)
print("Pandas version:", pd.__version__)
print("Shapely version:", shapely.__version__)

In [ ]:
lines_in = '../0_data/intermediate/4_lines.gpkg'
points_in = '../0_data/intermediate/4_lines_points.gpkg'

## 1. Read data

In [ ]:
lines = gpd.read_file( lines_in )
points = gpd.read_file( points_in )

In [ ]:
ax = lines.plot(color='lightgrey', zorder=-1)
points.plot(ax=ax, color='red', zorder=0)

# labels
for x, y, label in zip(points.geometry.x, points.geometry.y, points['point_id']):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")
# Note centroid will only work for line segments
for x, y, label in zip(lines.geometry.centroid.x, lines.geometry.centroid.y, lines['line_id']):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")

## 2. Create graph linking points via lines

In [ ]:
points_joining_lines = points.sjoin(lines)
points_joining_lines

In [ ]:
t = points_joining_lines[['line_id','point_id']].groupby('line_id').agg(list).reset_index()
t

In [ ]:
u = t.set_index('line_id')
u

In [ ]:
v = pd.DataFrame(u['point_id'].tolist(), index= u.index)
v

In [ ]:
n = nx.from_pandas_edgelist(v, 0, 1)

In [ ]:
n.edges

In [ ]:
type(n)

#### Convert to directed graph

In [ ]:
dn = n.to_directed()

In [ ]:
dn.edges

In [ ]:
ldn = nx.line_graph(dn)

In [ ]:
ldn.edges

In [ ]:
len(ldn.edges)

In [ ]:
ldn.nodes

In [ ]:
bc = nx.betweenness_centrality(ldn)
bc